In [ ]:
"""
To Make sure the Best Axon method works and also
to see if can decompose the axon any better 
(especially if can help with crosses and parts that aren't skeletonized             )

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [3]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-20 20:51:57,210 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-20 20:51:57,211 - settings - Setting database.user to celiib
INFO - 2021-02-20 20:51:57,212 - settings - Setting database.password to newceliipass
INFO - 2021-02-20 20:51:57,224 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-20 20:51:57,226 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-20 20:51:57,241 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-20 20:51:57,564 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-20 20:51:57,596 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-20 20:51:57,947 - settings - Setting enable_python_native_blobs to True


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu
import datajoint as dj
import neuron_visualizations as nviz
import neuron
import neuron_utils as nru
import proofreading_utils as pru
import classification_utils as clu
import system_utils as su
from importlib import reload

# Checking the Double Axon Neurons

In [5]:
multi_axon_segs,multi_axon_splits = (minnie.AutoProofreadNeurons2() & "cell_type_predicted='excitatory'"
 & "n_axons>1").fetch("segment_id","split_index")

In [6]:
len(multi_axon_segs)

9346

In [7]:
segment_id,split_index = (864691134947394812, 0) #was good for testing the high fidelity

In [8]:
curr_index = 
segment_id = multi_axon_segs[curr_index]
split_index = multi_axon_splits[curr_index]
segment_id,split_index

SyntaxError: invalid syntax (<ipython-input-8-6624d1fa8bc3>, line 1)

In [ ]:
du.plot_proofread_neuron(segment_id,
                        split_index=split_index,
                        plot_errored_faces=True,
                        plot_synapses=True,
                        plot_nuclei=True,
                        plot_axon=True)

In [ ]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

In [ ]:
if type(neuron_obj) == tuple:
    test_neuron = neuron_obj[0][split_index]
else:
    test_neuron = neuron_obj

In [ ]:
nviz.visualize_neuron(test_neuron,
                     limb_branch_dict="all")

In [ ]:
true_axons,true_axon_angles,axon_like_errors = clu.axon_classification(test_neuron,
                        return_axon_angles=True,
                       plot_candidates=False,
                       plot_axons=True,
                        best_axon=True,
                       verbose=True)

In [ ]:
# """
# Pseudocode: 
# if there even is an axon
# 1) Get the limb name of the best axon
# 2) Get the concept network of the limb and the starting node
# 3) Get the branches that would be axon
# 4) Delete the branches from the concept network
# 5) For each connected component in the leftover network
# - if not have starting node and all in axon-like: 
# add to list to add to the true axon

# 6) add the new nodes to the axon group
# """

# neuron_obj = test_neuron

# if len(best_limb_branch_dict) > 0:
    
#     #1) Get the limb name of the best axon
#     limb_name_of_axon = list(best_limb_branch_dict.keys())
#     if len(limb_name_of_axon) > 1:
#         raise Excpetion("More than 1 axon key")
    
#     limb_name_of_axon = limb_name_of_axon[0]
    
#     curr_limb = neuron_obj[limb_name_of_axon]
#     #2) Get the concept network of the limb and the starting node
#     conc_net = nx.Graph(curr_limb.concept_network)
#     start_node = curr_limb.current_starting_node
    
#     #3) Get the branches that would be axon
#     axon_branches = best_limb_branch_dict[limb_name_of_axon]
#     axon_like_branches = axon_like_limb_branch_dict[limb_name_of_axon]
    
#     #4) Delete the branches from the concept network
#     conc_net.remove_nodes_from(axon_branches)
    
    
#     #5) For each connected component in the leftover network
#     new_axon_branches = []
#     for conn_comp in nx.connected_components(conc_net):
#         conn_comp = list(conn_comp)
        
#         #- if not have starting node and all in axon-like: 
#         #add to list to add to the true axon
#         axon_like_in_conn_comp = np.intersect1d(axon_like_branches,conn_comp)
#         if start_node not in conn_comp and len(axon_like_in_conn_comp) == len(conn_comp):
#             new_axon_branches += conn_comp
    
    
        
#     if len(new_axon_branches) > 0:
#         best_limb_branch_dict[limb_name_of_axon] = np.array(list(axon_branches) + new_axon_branches)
        
# best_limb_branch_dict

# Getting a more high fidelity axon skeleton

In [ ]:
curr_index = 3010
segment_id = multi_axon_segs[curr_index]
split_index = multi_axon_splits[curr_index]
segment_id,split_index

In [ ]:
segment_id=864691135539149252
split_index = 0

In [ ]:
minnie.AutoProofreadNeurons2() & dict(segment_id = segment_id,
                                      split_index=split_index)

In [ ]:
du.plot_proofread_neuron(segment_id,
                        split_index=split_index,
                        plot_errored_faces=True,
                        plot_synapses=True,
                        plot_nuclei=True,
                        plot_axon=True)

In [ ]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

In [ ]:
if type(neuron_obj) == tuple:
    neuron_obj = neuron_obj[0][split_index]

In [ ]:
neuron_obj.segment_id

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

In [ ]:
print("hi")

In [ ]:
clu.axon_classification(neuron_obj,
                        plot_candidates=True,
                       best_axon=True,
                       plot_axons=True)

# Should not be absolutely tiny at the start

In [ ]:
"""
may have to worry about the width array

"""

In [ ]:
neuron_obj.axon_mesh, neuron_obj.axon_starting_coordinate

In [ ]:
verbose = True

axon_limb_name = neuron_obj.axon_limb_name
curr_limb = neuron_obj[axon_limb_name]
axon_starting_branch = neuron_obj.axon_starting_branch
axon_starting_branch_mesh = curr_limb[axon_starting_branch].mesh
axon_starting_coordinate = neuron_obj.axon_starting_coordinate

upstream_node_to_axon_starting_branch = None

if axon_starting_branch != curr_limb.current_starting_node:
    if verbose:
        print(f"Starting axon branch was not the limb starting node so generating border vertices and computing upstream branch")
    border_vertices_for_axon = tu.largest_border_to_coordinate(
        mesh = axon_starting_branch_mesh,
        coordinate =axon_starting_coordinate,
        distance_threshold = 1000,
        plot_border_vertices = False,
        error_on_no_border = True,
        plot_winning_border = False,
        verbose = False)
    
    upstream_node_to_axon_starting_branch = xu.upstream_node(curr_limb.concept_network_directional,
                                                             axon_starting_branch
                                                               )
    
else:
    if verbose:
        print(f"Starting axon branch was so using border vertices and NOT computing upstream branch")
    border_vertices_for_axon = curr_limb.current_touching_soma_vertices
    upstream_node_to_axon_starting_branch = None

soma_touching_vertices_dict = {0:[border_vertices_for_axon]}

if verbose:
    print(f"upstream_node_to_axon_starting_branch = {upstream_node_to_axon_starting_branch}")

In [ ]:
# divide the meshes into the only big continuous one (because if disconnected probably)
diff_meshes = tu.split_by_vertices(neuron_obj.axon_mesh)
axon_mesh_filtered = tu.filter_meshes_by_containing_coordinates(diff_meshes,
                                          nullifying_points=border_vertices_for_axon[0],
                                           method="distance",
                                           distance_threshold = 0,
                                          filter_away=False)[0]
if verbose:
    if len(diff_meshes) > 1:
        print(f"More than 1 seperated mesh (taking the top one): {diff_meshes}")

In [ ]:
limb_correspondence_individual,concept_network = pre.preprocess_limb(axon_mesh_filtered,
                   soma_touching_vertices_dict = soma_touching_vertices_dict,
                    meshparty_segment_size = 100,
                    combine_close_skeleton_nodes=True,
                    #combine_close_skeleton_nodes_threshold=1200,
                    combine_close_skeleton_nodes_threshold_meshparty = 1300,
                    filter_end_node_length_meshparty = 1500,
                    filter_end_node_length = 1500,
                                                                     
                    use_meshafterparty=False,
                    perform_cleaning_checks = True,
                    
                    
                    #concept_network parameters
                    run_concept_network_checks = True,
                    return_concept_network = True,
                    return_concept_network_starting_info=False,
                    
                    #printing controls
                    verbose = True,
                    print_fusion_steps=True,
                    
                    check_correspondence_branches = True,
                    filter_end_nodes_from_correspondence=True,
                    error_on_no_starting_coordinates=True,
                    invalidation_d=1200
                                                                     
                    
                   )

In [ ]:
limb_correspondence_individual[0].keys()

In [ ]:
concept_network
curr_concept_network = concept_network[0][0]
curr_concept_network

In [ ]:
starting_node = xu.get_starting_node(curr_concept_network,only_one=False)[0]

starting_coordinate = curr_concept_network.nodes[starting_node]["starting_coordinate"]
touching_soma_vertices = curr_concept_network.nodes[starting_node]["touching_soma_vertices"]

nviz.plot_limb_correspondence(limb_correspondence_individual,
                             scatters = [starting_coordinate],
                             scatter_size=1)

# nviz.plot_objects(skeletons=[k["branch_skeleton"] for k in limb_correspondence_individual.values()],
#                  scatters=[starting_coordinate],
#                  scatters_colors="red")

# Adding the part that connects back to the original mesh connection point

In [ ]:
"""
Psuedocode: 
1) Get the starting axon branch and the upstream node
2) Find the common endpoint
3) Add a skeletal branch from starting_coordinate (new)
and the common endpoint to the starting node of new
axon decomposition

"""
verbose = True
import numpy_utils as nu

#1) Get the starting axon branch and the upstream node
if upstream_node_to_axon_starting_branch is not None:
    print("Readjusting starting axon branch skeleton")
    
    #1) Get the starting axon branch and the upstream node
    axon_starting_branch,upstream_node_to_axon_starting_branch
    
    #2) Find the common endpoint
#     shared_endpoint = shared_skeleton_endpoints_for_connected_branches(neuron_obj[axon_limb_name],
#                                                 axon_starting_branch,
#                                                      upstream_node_to_axon_starting_branch,
#                                                 verbose=False)
    shared_endpoint = neuron_obj.axon_starting_coordinate
    
    if shared_endpoint.ndim > 1:
        shared_endpoint = shared_endpoint[0]
    
    #3) Add a skeletal branch from starting_coordinate (new)
    #and the common endpoint to the starting node of new
    #axon decomposition
    
    curr_endpoints = sk.find_skeleton_endpoint_coordinates(limb_correspondence_individual[starting_node]["branch_skeleton"])
    if len(nu.matching_rows(curr_endpoints,axon_starting_coordinate)) == 0:
        print("Fixing the axon starting branch endpoint to align with upstream branch")
        
        skeleton_pre_fix = limb_correspondence_individual[starting_node]["branch_skeleton"]
        new_skeleton_segment = np.array([starting_coordinate,shared_endpoint]).reshape(-1,2,3)

        limb_correspondence_individual[starting_node]["branch_skeleton"] = sk.stack_skeletons([skeleton_pre_fix,
                                                                                               new_skeleton_segment
                                                                                             ])
        if verbose:
            print(f"Starting Branch {starting_node} skeleton before fix: {skeleton_pre_fix.shape}")
            print(f"Starting Branch {starting_node} skeleton AFTER fix: {limb_correspondence_individual[starting_node]['branch_skeleton'].shape}")
    else:
        if verbose:
            print("Not attempting to fix the limb correspondence because the axon_starting_coordainte was already an endpoint")
    
    plot_connecting_skeleton_fix = False
    if plot_connecting_skeleton_fix:
        upstream_branch = neuron_obj[axon_limb_name][upstream_node_to_axon_starting_branch]
        nviz.plot_objects(meshes=meshes + [upstream_branch.mesh],
                          meshes_colors="random",
                          skeletons=skeletons +  [upstream_branch.skeleton],
                          skeletons_colors="random",
                                      scatters=[axon_starting_coordinate],
                                     scatter_size=0.3)
else:
    if verbose:
        print(f"Upstream node was None so don't have to adjust")

# MAY HAVE TO WORRY ABOUT COMPUTING THE WIDTH ARRAY

In [ ]:
nviz.plot_objects(meshes=[base_branch.mesh,neuron_obj["S0"].mesh],
                 skeletons=[base_branch.skeleton],
                 scatters=[neuron_obj.axon_starting_coordinate],
                 scatter_size=0.5)

In [ ]:
(base_final_skeleton,
    base_final_widths,
    base_final_seg_lengths)

In [ ]:
common_endpoint

In [ ]:
base_branch = neuron_obj[3][0]
common_endpoint = neuron_obj.axon_starting_coordinate

(base_final_skeleton,
    base_final_widths,
    base_final_seg_lengths) = nru.align_and_restrict_branch(base_branch,
                              common_endpoint=common_endpoint,
                             offset=0,
                             comparison_distance=2000,
                             skeleton_segment_size=1000,
                              verbose=True,
                             )
base_final_widths

In [ ]:
np.mean(base_final_widths)

In [ ]:
restr_skeleton = sk.restrict_skeleton_to_distance_from_coordinate(neuron_obj[0].skeleton,
                                                 neuron_obj[0].current_starting_coordinate,
                                                 10000,)
nviz.plot_objects(main_skeleton=neuron_obj[0].skeleton,
                 skeletons=[restr_skeleton],
                 skeletons_colors="red")          

In [ ]:
[k for k in dir(nru) if "endpoint" in k]

In [ ]:
branches

In [ ]:


axon_limb_dict = neuron_obj.axon_limb_branch_dict
axon_limb_name = list(axon_limb_dict.keys())[0]

limb_obj = neuron_obj[axon_limb_name]
branches = axon_limb_dict[axon_limb_name]

nru.closest_branch_endpoint_to_limb_starting_coordinate(limb_obj,
                                                    branches,
                                                       )


In [ ]:
neuron_obj.axon_starting_coordinate

In [ ]:
final_sk = sk.restrict_skeleton_from_start(neuron_obj[0].skeleton,
                                 cutoff_distance=10000)

In [ ]:
nviz.plot_objects(main_skeleton=neuron_obj[0].skeleton,
                 skeletons=[final_sk[0]],
                 skeletons_colors="red")          